In [192]:
import pandas as pd
import openpyxl
from statsmodels.formula.api import ols
import numpy as np
from scipy.stats import f

marketing = pd.read_excel("m.xlsx", index_col=0)
mod = ols(formula='sales ~ youtube + facebook + newspaper', data=marketing)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Mon, 13 Mar 2023   Prob (F-statistic):           1.58e-96
Time:                        14:55:20   Log-Likelihood:                -422.65
No. Observations:                 200   AIC:                             853.3
Df Residuals:                     196   BIC:                             866.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.5267      0.374      9.422      0.0

Dep. Variable:                  sales
To informacja wskazująca, która zmienna jest zależna.

Method:                 Least Squares
Informacja o metodzie - domyślnie metoda najmniejszych kwadratów.

No. Observations:                 200
Liczba obserwacji - w tym kontekście liczba wierszy (bez nagłówka).

Df Residuals:                     196
Liczba stopni swobody reszt: liczby obserwacji minus liczbę zmiennych niezależnych w modelu (w tym stałej). Innymi słowy, jest to liczba niezależnych obserwacji, które mogą być użyte do oszacowania reszt modelu.

Df Model:                           3
Liczba stopni swobody modelu, czyli liczba zmiennych objaśniających (niezależnych, bez stałej), które zostały uwzględnione w modelu.

Covariance Type:            nonrobust
Typ kowariancji użyty w estymacji macierzy wariancji-kowariancji parametrów modelu. W analizie regresji, estymacja macierzy kowariancji jest istotna, ponieważ pozwala na obliczenie błędów standardowych parametrów regresji, t-statystyk i przedziałów ufności.
'nonrobust': standardowa estymacja kowariancji, która nie uwzględnia żadnych specjalnych właściwości danych.
'HC0': metoda Hubera-White'a, która jest odporna na heteroskedastyczność i autokorelację reszt.
'HC1', 'HC2', 'HC3': różne wersje metody Hubera-White'a, które różnią się wagami reszt wykorzystanymi w estymacji macierzy kowariancji.
https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.get_robustcov_results.html

R-squared:                       0.897
Miara dopasowania modelu liniowego do danych i wskazuje, jaka część zmienności zmiennej objaśnianej jest wyjaśniona przez zmienne objaśniające.
R-kwadrat przyjmuje wartości od 0 do 1 i interpretuje się go jako procent wariancji zmiennej objaśnianej, która jest wyjaśniona przez model. Wartość R-kwadrat równe 0 oznacza, że model nie tłumaczy żadnej zmienności zmiennej objaśnianej, a wartość R-kwadrat równe 1 oznacza, że model tłumaczy całą zmienność zmiennej objaśnianej.

Należy jednak pamiętać, że wysoka wartość R-kwadrat nie zawsze oznacza, że model jest dobry. W szczególności, jeśli model ma wiele zmiennych objaśniających, wysoka wartość R-kwadrat może być wynikiem przeuczenia (overfittingu) modelu. Dlatego, warto zawsze uwzględnić także inne statystyki wyników analizy regresji, takie jak błędy standardowe parametrów, wartości t-statystyk i p-wartości.

$$R^2 = 1 - (SSR / SST)$$
$$R^2 = (SSM / SST)$$

In [193]:
yh = res.predict() # z modelu
y=np.array(marketing.iloc[:,3]) # dane do uczenia
ym=np.mean(y)
ssm=np.sum((yh-ym)**2)
sst=np.sum((y-ym)**2)
r2=ssm/sst
print(r2)
print(res.rsquared)

0.897210638178954
0.8972106381789522


Adj. R-squared:                  0.896
Skorygowane R-kwadrat jest miarą dopasowania modelu liniowego do danych, podobnie jak R-kwadrat, ale uwzględnia liczbę zmiennych objaśniających w modelu.
Skorygowane R-kwadrat jest skorygowaną wersją R-kwadrat i uwzględnia liczbę zmiennych objaśniających w modelu oraz liczbę obserwacji. Im większa liczba zmiennych objaśniających, tym większa szansa na przeuczenie (overfitting) modelu i sztucznie wyższy R-kwadrat. Skorygowane R-kwadrat koryguje tę niedoskonałość poprzez penalizowanie złożoności modelu i jest bardziej odpowiednią miarą dopasowania modelu dla modeli z wieloma zmiennymi objaśniającymi.
Skorygowane R-kwadrat przyjmuje wartości od 0 do 1 i interpretuje się go tak samo jak R-kwadrat - jako procent wariancji zmiennej objaśnianej, która jest wyjaśniona przez model. Wartość Skorygowanego R-kwadrat bliska 1 oznacza, że model dobrze wyjaśnia zmienność zmiennej objaśnianej, uwzględniając złożoność modelu.
Wartość Skorygowanego R-kwadrat może być wyższa lub niższa niż R-kwadrat, w zależności od liczby zmiennych objaśniających w modelu i liczby obserwacji. Dlatego warto zawsze uwzględnić obie statystyki w analizie wyników.

Skorygowany współczynnik determinacji:

$$\overline{R}^2=1 - (1 - R^2) \frac{n - 1}{n - p}$$

In [194]:
ro2=1-((1-r2)*(199/196))
print(ro2)
print(res.rsquared_adj)

0.8956373316204685
0.8956373316204668


F-test dla regresji wielowymiarowej

$$H_0: \qquad   \beta_1 = \beta_2 = \ldots = \beta_{p-1} = 0$$

$$H_1: \qquad  \beta_j \neq 0 \ \mathrm{dla \ co \ najmniej \ jednego} \ j.$$
Wyliczamy statystykę:

$$F=\frac{MSM}{MSE} = \frac{\mathrm{"wyjaśniona \ wariancja"}}{\mathrm{"niewyjaśniona \ wariancja"}} $$

In [195]:
ssm=np.sum((yh-ym)**2)
print(ssm)
ssr=np.sum((y-yh)**2)
print(ssr)
sst=np.sum((y-ym)**2)
print(sst)
print(ssm+ssr)
msm=ssm/3
print(msm)
mse=ssr/196
print(mse)
mst=sst/199
print(mst)
fi=msm/mse
print(fi)
print(res.fvalue)

6998.865821420864
801.8283785791496
7800.6942
7800.6942000000145
2332.9552738069547
4.090961115199742
39.199468341708545
570.2707036590954
570.2707036590942


Statystyka ta podlega rozkładowi F-Snedecora z $p-1$ i $n-p$ stopniami swobody. Ustalamy $\alpha=0,05$.

In [196]:
print(f.ppf(0.95, 3, 196))

2.6506765101121412


Jeśli wartość statystyki jest większa kwantylowi, odrzucamy hipotezę zerową. W przeciwnym wypadku przyjmujemy hipotezę zerową.

W naszym wypadku odrzucamy hipotezę zerową. Innymi słowy, odrzucamy hipotezę że wydatki na reklamy na poszczególne media nie mają wpływu na sprzedaż.

Obliczmy wartość $p$:

In [197]:
rv=f(3,196)
print(rv.sf(fi))
print(res.f_pvalue)

1.5752272560922012e-96
1.5752272560924532e-96


W naszym wypadku jest to "bliskie" zeru, więc możemy przyjąć, że się zgadza.

Jeśli $p\leqslant \alpha$ odrzucamy $H_0$ przyjmując $H_1$. W przeciwnym wypadku nie ma podstaw by odrzucić $H_0$.

Log-Likelihood:                -422.65
Log-Likelihood (logarytm funkcji wiarygodności) jest miarą dopasowania modelu do danych. Funkcja wiarygodności mierzy, jak dobrze model opisuje rozkład prawdopodobieństwa zmiennych objaśniających. Im większa wartość funkcji wiarygodności, tym lepiej model opisuje dane i lepiej pasuje do prawdziwego rozkładu danych. Logarytm funkcji wiarygodności jest ujemny, a większa wartość logarytmu oznacza lepsze dopasowanie modelu.
Log-Likelihood jest często stosowany do porównywania różnych modeli, ponieważ można porównać wartości logarytmów funkcji wiarygodności różnych modeli. Można również obliczyć test logarytmu funkcji wiarygodności (log-likelihood ratio test), aby ocenić, czy jeden model jest lepszy od drugiego.

Wzór:

$$\mathrm{Log-Likelihood} = -\frac{1}{2} \cdot n \cdot \ln(2 \pi) - \frac{1}{2} \cdot n\cdot  \ln(\sigma^2) - \frac{1}{2} \cdot \frac{SSR}{\sigma^2}$$

In [198]:
print(res.llf)

-422.645429303174


In [199]:
sigma2 = np.sum(res.resid ** 2) / 200 ## warto sprawdzić, że sigma2 a nie s2
print(sigma2)
n= 200
ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(sigma2) - 0.5 * ssr / sigma2
print(ll)

4.009141892895748
-422.645429303174


AIC:                             853.3
AIC (Akaike's Information Criterion) st miarą, która uwzględnia jakość dopasowania modelu do danych oraz złożoność modelu, czyli liczbę zmiennych objaśniających.
AIC mierzy, jak dobrze model opisuje dane, jednocześnie karząc za dodawanie dodatkowych zmiennych objaśniających do modelu, które mogą nie przynieść dodatkowej informacji. W przypadku AIC, mniejsza wartość oznacza lepszy model.
AIC jest wyrażony w postaci liczby punktów, gdzie mniejsza liczba oznacza lepszy model. AIC można wykorzystać do porównywania różnych modeli i wybierania modelu, który najlepiej pasuje do danych.

$$\operatorname{AIC} = -2 \cdot \operatorname{Log-Likelihood} + 2 p$$

In [200]:
print(res.aic)
aic = -2 * ll +2*4
print(aic)

853.290858606348
853.290858606348


BIC:                             866.5
BIC (Bayesian Information Criterion) mierzy jakość dopasowania modelu do danych oraz jego złożoność, ale uwzględnia również liczbę obserwacji w próbie.
BIC jest miarą, która jest oparta na teorii Bayesa i opiera się na koncepcji, że lepszy model jest bardziej prawdopodobny zgodnie z zasadą największego prawdopodobieństwa, ale powinien też mieć mniejszą liczbę parametrów.
BIC jest wyrażony w postaci liczby punktów, gdzie mniejsza wartość oznacza lepszy model. BIC można wykorzystać do porównywania różnych modeli i wybierania modelu, który najlepiej pasuje do danych.

$$\operatorname{BIC} = -2 \cdot \operatorname{Log-Likelihood} + p\cdot \ln(n)$$

In [201]:
print(res.bic)
bic = -2 * ll + 4 * np.log(200)
print(bic)

866.4841280725402
866.4841280725402


Omnibus:                       60.414
Omnibus to test normalności reszt w modelu regresji, który bada, czy reszty w modelu są rozłożone normalnie.
Test Omnibus jest testem chi-kwadrat i ma dwie hipotezy:
Hipoteza zerowa: Reszty w modelu mają rozkład normalny.
Hipoteza alternatywna: Reszty w modelu nie mają rozkładu normalnego.

Do ew. dokończenia (??).

In [202]:
from scipy import stats

skewness = stats.skew(res.resid)
kurtosis = stats.kurtosis(res.resid)
print(skewness)
print(kurtosis)

omnibus, p_value = stats.normaltest(res.resid)

# Wyświetlenie wyników
print('Omnibus: ', omnibus)
print('P-value: ', p_value)

-1.3273536543621196
3.3318912890624732
Omnibus:  60.41395945525499
P-value:  7.60808558322375e-14


Durbin-Watson:                   2.084
Durbin-Watson to statystyka diagnostyczna, która służy do oceny autokorelacji reszt w modelu regresji liniowej. Autokorelacja reszt występuje, gdy występuje zależność między wartościami reszt w kolejnych obserwacjach.
Statystyka Durbin-Watson przyjmuje wartości od 0 do 4 i pozwala na ocenę występowania autokorelacji reszt w modelu regresji. Wartość 2 oznacza brak autokorelacji, wartości poniżej 2 wskazują na występowanie dodatniej autokorelacji, a wartości powyżej 2 wskazują na występowanie ujemnej autokorelacji.

Interpretacja wyniku Durbin-Watson może być następująca:

Wartość Durbin-Watson bliska 2 wskazuje na brak autokorelacji reszt.
Wartość Durbin-Watson mniejsza od 2 wskazuje na występowanie dodatniej autokorelacji reszt (czyli wartości reszt w kolejnych obserwacjach są skorelowane dodatnio).
Wartość Durbin-Watson większa od 2 wskazuje na występowanie ujemnej autokorelacji reszt (czyli wartości reszt w kolejnych obserwacjach są skorelowane ujemnie).

$$DW = \sum_{i=2}^n [(r_i - r_{i-1})^2] / \sum_{i=1}^n (r_i)^2$$

In [203]:
from statsmodels.stats.stattools import durbin_watson

dw = durbin_watson(res.resid)
print(dw)

e=res.resid
diff_e = np.diff(e)
sum_sq_diff_e = np.sum(diff_e ** 2)
sum_sq_e = np.sum(e ** 2)
dw2 = sum_sq_diff_e / sum_sq_e
print(dw2)

2.083648405294408
2.083648405294408


Jarque-Bera (JB):              151.241
arque-Bera (JB) to test normalności rozkładu reszt w modelu regresji. Test Jarque-Bera wykorzystuje skośność (skewness) i kurtozę (kurtosis) rozkładu reszt, aby sprawdzić, czy rozkład ten jest zbliżony do rozkładu normalnego.
Wynik testu Jarque-Bera to wartość chi-kwadrat (χ²) z dwoma stopniami swobody i odpowiada na pytanie, czy rozkład reszt w modelu regresji jest zbliżony do rozkładu normalnego. Wartość p-value dla testu Jarque-Bera pozwala na określenie, czy odrzucamy hipotezę o normalności rozkładu reszt (mała wartość p-value) czy nie (wysoka wartość p-value).
Interpretacja wyniku testu Jarque-Bera może być następująca:
Wartość chi-kwadrat bliska zeru i wysoka wartość p-value wskazują na brak istotnych odchyleń od rozkładu normalnego i odrzucenie hipotezy o niestandardowym rozkładzie reszt.
Wartość chi-kwadrat dużo większa od zera i niska wartość p-value wskazują na duże odchylenie od rozkładu normalnego i odrzucenie hipotezy o normalności rozkładu reszt.

$$JB = n/6 \cdot (S^2 + 1/4 \cdot (K - 3)^2)$$
$$JB = n/6 \cdot (S^2 + 1/4 \cdot K^2)$$

In [204]:
from statsmodels.stats.stattools import jarque_bera
from scipy.stats import chi2

jb_stat, jb_pvalue, _, _ = jarque_bera(res.resid)
print(jb_stat)
print(jb_pvalue)

n = 200
JB = n / 6 * (skewness**2 + 1/4 * kurtosis **2)

# wyznaczmy wartość krytyczną dla testu Jarque-Bera (z poziomem istotności alpha = 0.05 i dwoma stopniami swobody)
alpha = 0.05
df = 2
JB_critical = chi2.ppf(1-alpha, df)
pvalue = 1 - chi2.cdf(JB, df)
print(JB)
print(pvalue)

151.2414204760357
1.4399347942411259e-33
151.2414204760357
0.0


Cond. No.                         545.
Cond. No. to liczba uwarunkowania macierzy w modelu regresji. Liczba uwarunkowania odzwierciedla stopień, w jakim zmiany w danych wejściowych (zmienne niezależne) wpływają na zmienność wyjścia (zmienna zależna) oraz na stabilność i dokładność wyników modelu regresji.

Wartość Cond. No. wynosi zwykle od 1 (idealna sytuacja) do bardzo wysokich wartości (np. 100 lub więcej), co oznacza, że macierz w modelu regresji jest źle uwarunkowana i może prowadzić do problemów numerycznych lub niestabilności wyników.

Liczba uwarunkowania jest szczególnie istotna w przypadku wielu zmiennych niezależnych lub silnie skorelowanych zmiennych niezależnych. W takich przypadkach zmiany w jednej zmiennej mogą prowadzić do znaczących zmian w innych zmiennych, co wpływa na stabilność wyników modelu regresji.

Interpretacja wartości Cond. No. może być następująca:

Wartość Cond. No. bliska 1 wskazuje na dobrze uwarunkowaną macierz i stabilne wyniki modelu regresji.
Wartość Cond. No. większa od 1, ale mniejsza niż 100, wskazuje na pewne problemy z uwarunkowaniem macierzy, ale wyniki modelu regresji są wciąż dość stabilne.
Wartość Cond. No. większa niż 100 wskazuje na poważne problemy z uwarunkowaniem macierzy i niestabilność wyników modelu regresji.

$$Cond. No. = max(s) / min(s)$$

s- wartości osobliwe macierzy

Wartość osobliwa macierzy (singular value) to pojęcie z algebry liniowej odzwierciedlające "siłę" wektora w macierzy. W kontekście modelu regresji liniowej wartości osobliwe macierzy X określają wpływ poszczególnych zmiennych niezależnych na zmienność wyjścia (zmienną zależną) oraz umożliwiają wyznaczenie wartości Cond. No. macierzy X.

Wartość osobliwa macierzy jest równa pierwiastkowi kwadratowemu z wartości własnych macierzy X.T * X lub X * X.T, z których jedna jest macierzą kowariancji, a druga macierzą korelacji. Wartości osobliwe macierzy są dodatnie i posortowane malejąco.

In [205]:
one=np.ones((200,1))
x=np.array(marketing.iloc[:,0:3])
x=np.append(one,x,axis=1)
print(np.linalg.cond(x))
singular = np.linalg.svd(x, compute_uv=False)
print(singular)
cond_no = np.max(singular) / np.min(singular)
print(cond_no)

545.2331163467246
[2946.18689305  478.23029185  233.39770042    5.40353622]
545.2331163467246


In [206]:
C = np.dot(x.T, x)
eigenvalues, eigenvectors = np.linalg.eig(C)
singular_values = np.sqrt(eigenvalues)
singular_values_sorted = np.sort(singular_values)[::-1]
print(singular_values_sorted)

[2946.18689305  478.23029185  233.39770042    5.40353622]
